# Aula 7

## Tokenizadores

### Tokenizador por Espaço

Desenvolvido com base no principal delimitador para uma grande parcela das línguas naturais humanas: o espaço

In [15]:
texto = "No meio do caminho tinha uma pedra."
texto.split()

['No', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra.']

### Tokenizador baseado numa expressão regular

Segmenta as palavras de um texto com base em delimitadores como espaço, pontuações e início/fim de uma sequência (\b)

### Tokenizador baseado em Regras

1. Buscar todas as ocorrências de valores numéricos e financeiros (R$1,00; $46; etc.)

2. Buscar todas as ocorrências de sequências de 1 ou mais caracteres

3. Buscar todas as ocorrências de sequências sem espaço


### Tokenizador baseado em Regras do NLTK

In [16]:
import nltk
from tokenizers import Tokenizer

# nltk.download('punkt')

versos = """O menino jogou bola ontem às 16:00."""

nltk.word_tokenize(versos, language='portuguese')

['O', 'menino', 'jogou', 'bola', 'ontem', 'às', '16:00', '.']

In [17]:
text = """Hello everyone!!!"""

nltk.word_tokenize(text, language='english')

['Hello', 'everyone', '!', '!', '!']

### Byte-Pair Encoding (BPE)

Inicializando o tokenizador

In [18]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

Inicializando o Módulo de Treinamento

Define-se um vocabulário desejado com 30000 símbolos

In [19]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], 
                     vocab_size=30000, 
                     min_frequency=0,
                     continuing_subword_prefix="##")

Definindo pré-tokenizador por espaço

In [20]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

Treinando o tokenizador

In [21]:
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

Salvando o Tokenizador

In [22]:
tokenizer.save("tokenizer-wiki.json")

Carregando o tokenizador

In [23]:
tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

Tokenizando textos

In [24]:
texto = "I don't go out tonight."
output = tokenizer.encode(texto)
output.tokens

['I', 'don', "'", 't', 'go', 'out', 'ton', '##ight', '.']

In [11]:
output.ids

[45, 11322, 11, 88, 9382, 9031, 14432, 8914, 18]

In [25]:
tokenizer.decode(output.ids)

"I don ' t go out ton ##ight ."

### Byte-level BPE

Inicializando o tokenizador e o córpus de treinamento (Wikipedia)

In [35]:
from tokenizers import ByteLevelBPETokenizer

files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]

tokenizer = ByteLevelBPETokenizer()

Treinando o tokenizador

In [36]:
tokenizer.train(files=files, vocab_size=52000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

Salvando o tokenizador

In [37]:
tokenizer.save("tokenizer-wiki2.json")

Tokenizando um texto

In [38]:
output = tokenizer.encode("Eu estou na aula de Mineração de Dados.")
output.tokens

['E',
 'u',
 'Ġest',
 'ou',
 'Ġna',
 'Ġa',
 'ula',
 'Ġde',
 'ĠMin',
 'era',
 'Ã§',
 'Ã£o',
 'Ġde',
 'ĠDad',
 'os',
 '.']

### Unigram

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram
from tokenizers.trainers import UnigramTrainer

files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]

# inicializando o tokenizador
tokenizer = Tokenizer(Unigram())
# inicilizando o módulo de treinamento
trainer = UnigramTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], 
                     vocab_size=30000, 
                     min_frequency=0,
                     continuing_subword_prefix="##")
# treinando o tokenizador
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [ ]:
tokenizer.save("tokenizer-wiki3.json")

## Capitalização

Processo de colocar os tokens em letra minúscula para normalização do texto.

In [1]:
import nltk
# nltk.download('punkt')

versos = """No meio do caminho tinha uma pedra
Tinha uma pedra no meio do caminho""".lower()

nltk.word_tokenize(versos, language='portuguese')

['no',
 'meio',
 'do',
 'caminho',
 'tinha',
 'uma',
 'pedra',
 'tinha',
 'uma',
 'pedra',
 'no',
 'meio',
 'do',
 'caminho']

### Tokenização de Sentenças

In [2]:
import nltk
# nltk.download('punkt')

texto = 'Eu estou na aula de Mineração de Dados. Os estudantes são muito bons.'

nltk.sent_tokenize(texto, language='portuguese')

['Eu estou na aula de Mineração de Dados.', 'Os estudantes são muito bons.']

## Lematização

Instalando o Spacy

In [4]:
!pip install spacy
!python3 -m spacy download pt_core_news_lg


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: C:\Users\RooneyCoelho\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
     -------------------------------------- 568.2/568.2 MB 1.7 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: C:\Users\RooneyCoelho\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [5]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("pt_core_news_lg")

In [6]:
doc = nlp("O passado é só uma história que nos contamos.")

for token in doc:
  print(token, token.lemma_)

O o
passado passado
é ser
só só
uma um
história história
que que
nos nós
contamos contar
. .


## Radicalização

In [7]:
import nltk
# nltk.download('rslp')

raiz = nltk.stem.RSLPStemmer()

tokens = nltk.word_tokenize('A comida estava gostosa', language='portuguese')
[raiz.stem(token) for token in tokens]

['a', 'com', 'est', 'gost']

# Modelos de Linguagem e N-gramas

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RooneyCoelho\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

### Passo 1: Carregando o Córpus

In [9]:
texto = """No meio do caminho tinha uma pedra
Tinha uma pedra no meio do caminho
Tinha uma pedra
No meio do caminho tinha uma pedra"""

texto = texto.lower().split('\n')
texto

['no meio do caminho tinha uma pedra',
 'tinha uma pedra no meio do caminho',
 'tinha uma pedra',
 'no meio do caminho tinha uma pedra']

### Passo 2: Tokenizando as Sentenças do Córpus

In [10]:
texto_tok = []
for verso in texto:
  tokens = nltk.word_tokenize(verso, language='portuguese')
  texto_tok.append(tokens)

texto_tok

[['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra'],
 ['tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho'],
 ['tinha', 'uma', 'pedra'],
 ['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra']]

### Passo 3: Inserindo Marcadores de Início e Fim de Sentença
Suponha que queiramos definir um modelo de linguagem com bigramas, ou seja, calcular as chances de uma palavra com base na anterior (e.g., $P(pedra | uma)$, temos que marcar o início e fim da sentença para poder prever as changes da primeira palavra (e.g., $P(no | \langle s \rangle)$) e o fim da sentença ((e.g., $P(\langle/ s \rangle)$ | pedra)). Este processo é conhecido como *padding*.

Podemos fazer o *padding* de uma sentença utilizando o método **nltk.lm.preprocessing.pad_both_ends**:

In [11]:
from nltk.lm.preprocessing import pad_both_ends

ngramas = 2 # definindo o número de n-gramas (no caso, 2 -> bigramas)

texto_tok_pad = []
for verso in texto_tok:
  padded = pad_both_ends(verso, n=ngramas)
  texto_tok_pad.append(list(padded))

texto_tok_pad

[['<s>', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '</s>'],
 ['<s>', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', '</s>'],
 ['<s>', 'tinha', 'uma', 'pedra', '</s>'],
 ['<s>', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '</s>']]

### Passo 4: Calculando os N-Gramas

Uma vez que as sentenças do nosso córpus foram pré-processadas, podemos calcular os n-gramas (neste caso, os bigramas), utilizando o método **nltk.ngrams**:

In [12]:
ngramas = 2

bigramas_pad = []
for verso in texto_tok_pad:
  bigramas = nltk.ngrams(verso, ngramas)
  bigramas_pad.append(list(bigramas))

bigramas_pad

[[('<s>', 'no'),
  ('no', 'meio'),
  ('meio', 'do'),
  ('do', 'caminho'),
  ('caminho', 'tinha'),
  ('tinha', 'uma'),
  ('uma', 'pedra'),
  ('pedra', '</s>')],
 [('<s>', 'tinha'),
  ('tinha', 'uma'),
  ('uma', 'pedra'),
  ('pedra', 'no'),
  ('no', 'meio'),
  ('meio', 'do'),
  ('do', 'caminho'),
  ('caminho', '</s>')],
 [('<s>', 'tinha'), ('tinha', 'uma'), ('uma', 'pedra'), ('pedra', '</s>')],
 [('<s>', 'no'),
  ('no', 'meio'),
  ('meio', 'do'),
  ('do', 'caminho'),
  ('caminho', 'tinha'),
  ('tinha', 'uma'),
  ('uma', 'pedra'),
  ('pedra', '</s>')]]

Contudo, para deixar nosso modelo de linguagem mais robusto, vamos calcular os **unigramas** além dos **bigramas** utilizando o comando **nltk.util.everygrams**:

In [13]:
from nltk.util import everygrams
ngramas = 2

ngramas_pad = []
for verso in texto_tok_pad:
  bigramas = everygrams(verso, max_len=ngramas)
  ngramas_pad.append(list(bigramas))

ngramas_pad

[[('<s>',),
  ('<s>', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)]]

### Passo 5: Colocando todos os tokens do córpus numa única lista
**nltk.lm.preprocessing.flatten**:

Este método converte junta os elementos de sublistas em uma única lista. Por exemplo:

```python
>>> lista = [[1, 2], [3, 4]]
>>> flatten(lista)
[1, 2, 3, 4]
```

Como pode ser visto abaixo, nós o utilizamos para juntar todas os tokens das sentenças de nosso corpus numa única lista.

In [15]:
texto_tok_pad

[['<s>', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '</s>'],
 ['<s>', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', '</s>'],
 ['<s>', 'tinha', 'uma', 'pedra', '</s>'],
 ['<s>', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '</s>']]

In [14]:
from nltk.lm.preprocessing import flatten

tokens = list(flatten(texto_tok_pad)) # juntando as palavras do nosso córpus

tokens

['<s>',
 'no',
 'meio',
 'do',
 'caminho',
 'tinha',
 'uma',
 'pedra',
 '</s>',
 '<s>',
 'tinha',
 'uma',
 'pedra',
 'no',
 'meio',
 'do',
 'caminho',
 '</s>',
 '<s>',
 'tinha',
 'uma',
 'pedra',
 '</s>',
 '<s>',
 'no',
 'meio',
 'do',
 'caminho',
 'tinha',
 'uma',
 'pedra',
 '</s>']

### Passo 6: Definindo o Vocabulário

**nltk.lm.Vocabulary**

Utilizado para definir o vocabulário do nosso córpus. Recebe dois parâmetros como entrada: uma lista com todos os tokens do nosso córpus e a variável *unk_cutoff*, a qual passa a considerar palavras abaixo de um limiar de frequência como palavras fora do vocabuário.

In [16]:
from nltk.lm import Vocabulary

vocab = Vocabulary(tokens, unk_cutoff=1) # definindo o vocabulário do nosso córpus

Obtendo as frequências das palavras do córpus com o comando *counts*

In [18]:
vocab.counts

Counter({'<s>': 4,
         'no': 3,
         'meio': 3,
         'do': 3,
         'caminho': 3,
         'tinha': 4,
         'uma': 4,
         'pedra': 4,
         '</s>': 4})

procurando uma palavra no vocabulário. Caso não encontrada, o token de palavra fora do vocabulário será retornada (\<UNK>)

In [19]:
vocab.lookup("tinha"), vocab.lookup("homem") 

('tinha', '<UNK>')

## Simplificando o Pré-processamento

Agora que você sabe cada passo do pré-processamento (inserir marcadores de início e fim de sentença, calcular os n-gramas, juntar todos os tokens do corpus numa lista e definir o vocabulário), este processo pode ser simplificado pela funcionalidade **nltk.lm.preprocessing.padded_everygram_pipeline**:

In [20]:
from nltk.lm.preprocessing import padded_everygram_pipeline

ngramas = 2

ngramas_pad, vocab = padded_everygram_pipeline(ngramas, texto_tok)

ngramas_pad = [list(w) for w in ngramas_pad]
ngramas_pad

[[('<s>',),
  ('<s>', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)]]

### Passo 7: Treinando um modelo de linguagem

Um modelo de linguagem pode ser treinado utilizando a funcionalidade **nltk.lm.MLE**

In [21]:
from nltk.lm.preprocessing import padded_everygram_pipeline, flatten
from nltk.lm import MLE

ngramas = 2
ngramas_pad, vocab = padded_everygram_pipeline(ngramas, texto_tok)
lm = MLE(ngramas)
lm.fit(ngramas_pad, vocab)

Dado o token **\<s>**, gerando um texto de 4 tokens com o modelo de linguagem treinado.

In [22]:
lm.generate(4, text_seed=["<s>"])

['no', 'meio', 'do', 'caminho']

Probabilidade da palavra *no*:

In [23]:
print("Probabilidade comum: ", lm.score("no"))
print("Probabilidade logarítmica: ", lm.logscore("no"))

Probabilidade comum:  0.09375
Probabilidade logarítmica:  -3.415037499278844


Probabilidade da palavra *tinha* dado a palavra *caminho*:

In [24]:
print("Probabilidade comum: ", lm.score("tinha", context=["caminho"]))
print("Probabilidade logarítmica: ", lm.logscore("tinha", context=["caminho"]))

Probabilidade comum:  0.6666666666666666
Probabilidade logarítmica:  -0.5849625007211563


## Representações Vetoriais para Palavras, Sentenças e Documentos

### Similaridade por Cossenos

Normalmente em PLN, a distância entre dois vetores é calculada através da similaridade por cosseno.

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

po = np.array([[5, 10]])
mestre_tigresa = np.array([[7.5, 2.5]])

cosine_similarity(po, mestre_tigresa)[0][0]

0.7071067811865475

## Representação *One-Hot*

Palavras e documentos são representados por vetores de dimensão do tamanho do vocabulário. Os vetores assumem valores binários (0 ou 1) 

In [29]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

X = [["no"], ["meio"], ["do"], ["caminho"], ["tinha"], ["uma"], ["pedra"]]

enc.fit(X)
vocab = list(enc.categories_[0])
vetores = enc.transform(X).toarray()

print('Vocabulário: ', vocab)
print('Vetores')
vetores

Vocabulário:  ['caminho', 'do', 'meio', 'no', 'pedra', 'tinha', 'uma']
Vetores


array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.]])

Vetor One-Hot de *pedra*

In [30]:
vetores[vocab.index('pedra')]

array([0., 0., 0., 0., 0., 1., 0.])

## Matriz de Frequência Termo-Documento

Dado um vocabulário e um conjunto de documentos, as representações das palavras e dos documentos podem ser calculadas a partir da contagem de cada palavra em cada documento.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['no meio do caminho tinha uma pedra',
 'tinha uma pedra no meio do caminho',
 'tinha uma pedra',
 'no meio do caminho tinha uma pedra']

vectorizer = CountVectorizer()

vetores = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names_out()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(vetores.toarray())

Vocabulário
['caminho' 'do' 'meio' 'no' 'pedra' 'tinha' 'uma']

Matrix
[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [0 0 0 0 1 1 1]
 [1 1 1 1 1 1 1]]


## Matriz Termo-Termo

Dado um vocabulário, a representação de uma palavra pode ser calculada a partir da contagem de sua co-ocorrência com cada palavra do vocabulário num determinado contexto (e.g. documento, sentença, etc.).

In [33]:
corpus = ['no meio do caminho tinha uma pedra',
 'tinha uma pedra no meio do caminho',
 'tinha uma pedra',
 'no meio do caminho tinha uma pedra']

corpus_tok = [verso.split() for verso in corpus]

vocab = ["no", "meio", "do", "caminho", "tinha", "uma", "pedra"]
vetores = np.zeros((len(vocab), len(vocab)))

for verso in corpus_tok:
  for i, w1 in enumerate(vocab):
    for j, w2 in enumerate(vocab):
      if i != j:
        if w1 in verso and w2 in verso:
          vetores[i, j] += 1

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(vetores)

Vocabulário
['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra']

Matrix
[[0. 3. 3. 3. 3. 3. 3.]
 [3. 0. 3. 3. 3. 3. 3.]
 [3. 3. 0. 3. 3. 3. 3.]
 [3. 3. 3. 0. 3. 3. 3.]
 [3. 3. 3. 3. 0. 4. 4.]
 [3. 3. 3. 3. 4. 0. 4.]
 [3. 3. 3. 3. 4. 4. 0.]]


## Remoção de Palavras Vazias

Palavras vazias (e.g., artigos, preposições, etc.), que possuem alta frequência em todos os documentos, podem ser removidas da contagem para melhorar a distinção entre documentos

In [36]:
import nltk
#nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

from sklearn.feature_extraction.text import CountVectorizer

corpus = ['no meio do caminho tinha uma pedra',
 'tinha uma pedra no meio do caminho',
 'tinha uma pedra',
 'no meio do caminho tinha uma pedra']

vectorizer = CountVectorizer(stop_words=stopwords)

vetores = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names_out()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(vetores.toarray())

Vocabulário
['caminho' 'meio' 'pedra']

Matrix
[[1 1 1]
 [1 1 1]
 [0 0 1]
 [1 1 1]]


## TF-IDF

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

corpus = ['ainda que mal pergunte',
 'ainda que mal respondas',
 'ainda que mal te entenda',
 'ainda que mal repitas']

vectorizer = Pipeline([('count', CountVectorizer()),
                 ('tfid', TfidfTransformer())])

vetores = vectorizer.fit_transform(corpus)
vocab = vectorizer['count'].get_feature_names_out()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(np.round(vetores.toarray(), 2))

Vocabulário
['ainda' 'entenda' 'mal' 'pergunte' 'que' 'repitas' 'respondas' 'te']

Matrix
[[0.39 0.   0.39 0.74 0.39 0.   0.   0.  ]
 [0.39 0.   0.39 0.   0.39 0.   0.74 0.  ]
 [0.31 0.6  0.31 0.   0.31 0.   0.   0.6 ]
 [0.39 0.   0.39 0.   0.39 0.74 0.   0.  ]]


Acessando o primeiro (*ainda que mal pergunte*) e terceiro (*ainda que mal te entenda*) versos e calculando a similaridade entre eles.

In [39]:
verso1 = vetores[0, :]
verso3 = vetores[2, :]

cosine_similarity(verso1, verso3)[0][0]

0.3611073242896012

## Word Embeddings

https://projector.tensorflow.org/

In [ ]:
!wget http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s50.zip

In [43]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('cbow_s50.txt')

Acessando o word embedding da palavra *menino*

In [46]:
word2vec['menino']

array([ 0.047754, -0.190243,  0.290581,  0.035822,  0.2301  , -0.139099,
       -0.232351, -0.119084,  0.327645,  0.160017, -0.5318  ,  0.093309,
       -0.545777, -0.166715,  0.044872, -0.094386, -0.017529, -0.053898,
        0.189092, -0.233779, -0.302459,  0.707696, -0.146762,  0.258651,
        0.25436 , -0.071892,  0.132296, -0.072721,  0.162642,  0.348834,
        0.129191, -0.030967,  0.048024,  0.26683 , -0.076066,  0.352168,
        0.629779, -0.403468, -0.473612,  0.456509,  0.008285,  0.066872,
        0.082632, -0.128989,  0.107645,  0.119981,  0.219388, -0.141599,
       -0.20074 , -0.30657 ], dtype=float32)

Palavras mais semelhantes ao verbo *estudar*

In [48]:
word2vec.most_similar('estudar')

[('pesquisar', 0.8674625158309937),
 ('ensinar', 0.8485607504844666),
 ('leccionar', 0.8399008512496948),
 ('moldar', 0.8285380601882935),
 ('desenvolver', 0.8203483819961548),
 ('focalizar', 0.8188527226448059),
 ('cursar', 0.8175848126411438),
 ('projectar', 0.8173723816871643),
 ('desenhar', 0.8155599236488342),
 ('enriquecer', 0.8142802715301514)]

Similaridade por cosseno entre os word embeddings das palavras *menino* e *cachorro*

In [49]:
word2vec.similarity('menino', 'cachorro')

0.8441181

Inferência lógica para: *odiar* está para *odiando*, assim como *amar* está para...

In [50]:
word2vec.most_similar(positive=['amar', 'odiando'], negative=['odiar'])

[('amando', 0.7472066283226013),
 ('desperto', 0.7231095433235168),
 ('quieto', 0.6835169196128845),
 ('tranqüilo', 0.6812532544136047),
 ('surdo', 0.6798273921012878),
 ('louco', 0.6784767508506775),
 ('quieta', 0.6757060885429382),
 ('sã³brio', 0.6748781204223633),
 ('rouco', 0.6719405651092529),
 ('sossegado', 0.6716687679290771)]